In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
file_path = '/kaggle/input/cic-iot-2023/part-00002-363d1ba3-8ab5-4f96-bc25-4d5862db7cb9-c000.csv'
df = pd.read_csv(file_path)



# Display the first few rows of the DataFrame and the summary information

df_head = df.head()

df_info = df.info()

df_head, df_info
df.head()


# Set the style for seaborn plots
sns.set_style('whitegrid')

# Increase default font size for readability
sns.set_context('talk')

In [ ]:
# Check for missing values
missing_values = df.isnull().sum()

# Check for duplicate rows
duplicates = df.duplicated().sum()

missing_values, duplicates


In [ ]:
# Descriptive statistics for the numerical features

numerical_stats = df.describe()



# Distribution of attack labels

attack_label_counts = df['label'].value_counts()



numerical_stats, attack_label_counts

Descriptive Statistics

    Count: There are 275,258 entries for each numerical feature.
    Mean and Std (Standard Deviation): We can see the average values for each feature along with the spread of the data. For instance, the flow_duration has a high standard deviation relative to its mean, indicating large variations in flow duration.
    Min and Max: These values show us the range of the data for each feature. Some features have a wide range while others are more concentrated.
    25%, 50% (Median), and 75%: These percentiles give us a sense of the distribution of the data.

Attack Label Distribution

The label counts show the frequency of each type of attack in the dataset:

    The most common attack type is DDoS-ICMP_Flood with 42,340 occurrences, followed by DDoS-UDP_Flood.
    The least common are web-based attacks like SqlInjection, CommandInjection, XSS, and Backdoor_Malware, which have very few occurrences in this particular CSV file.
    There is also benign traffic labeled as BenignTraffic, which is important for training classification models to distinguish between normal and malicious traffic.

In [ ]:


# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Bar chart for the distribution of attack labels
plt.figure(figsize=(14, 8))
attack_label_counts.plot(kind='bar')
plt.title('Distribution of Attack Labels')
plt.xlabel('Attack Type')
plt.ylabel('Frequency')
plt.xticks(rotation=90)  # Rotate the x labels to show them clearly
plt.tight_layout()  # Adjust the plot to ensure everything fits without overlapping

# Show the plot
plt.show()


The bar chart above visualizes the frequency of each attack type in the dataset. As we can see, some attack types like DDoS-ICMP_Flood, DDoS-UDP_Flood, and others in the DDoS category are much more frequent than others.

In [ ]:
# Selecting a few numerical features for histograms
features_to_plot = ['flow_duration', 'Rate', 'Protocol Type']

# Plot histograms for the selected features
plt.figure(figsize=(15, 5))

for i, feature in enumerate(features_to_plot, 1):
    plt.subplot(1, 3, i)
    sns.histplot(df[feature], bins=50, kde=False)
    plt.title(f'Distribution of {feature}')
    plt.xlabel(feature)
    plt.ylabel('Frequency')

plt.tight_layout()
plt.show()


The histograms for flow_duration, Rate, and Protocol Type show the following:

    flow_duration: Most of the flow durations are concentrated at the lower end, near zero, suggesting that many flows are very short-lived, which is characteristic of some types of attacks that generate a lot of quick traffic.
    Rate: This feature also shows a heavy concentration near zero, indicating that many flows have low rates, with a few exceptions going up to higher values.
    Protocol Type: The distribution is a bit more spread out, but there is a clear concentration around certain protocol numbers, likely corresponding to common protocols like TCP and UDP.

In [ ]:
# Identify categorical variables
categorical_columns = df.select_dtypes(include=['object']).columns

# Apply one hot encoding to the categorical columns
df_encoded = pd.get_dummies(df, columns=categorical_columns)

# Sample a fraction of the data to avoid memory issues
df_sampled = df_encoded.sample(frac=0.1, random_state=1)

# Calculate the correlation matrix
corr_subset = df_sampled.corr()

# Plot the heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr_subset, annot=False, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .5})
plt.title('Correlation Matrix Heatmap')
plt.show()


The heatmap above displays the correlation matrix for a subset of the data, which provides a visual representation of the correlation coefficients between pairs of features. In such a large dataset with many features, correlations can reveal:

    Strong positive or negative relationships: Highly correlated features can sometimes be redundant, and one of the pair might be excluded to simplify the model.
    Lack of relationship: Features that have low correlation with others might be unique contributors to the model or may not contribute at all.

For features that are strongly correlated with the attack labels, those would be considered important for detecting and classifying attacks.

In [ ]:
# Protocol-based analysis: Let's see the distribution of attack types across different protocol types
# We'll group the data by 'Protocol Type' and see the count of each attack label within each protocol

# Grouping the data
protocol_attack_distribution = df.groupby('Protocol Type')['label'].value_counts().unstack(fill_value=0)

# Because there might be a large number of protocols, let's visualize the distribution for the top protocols only
top_protocols = df['Protocol Type'].value_counts().head(10).index
protocol_attack_distribution_top = protocol_attack_distribution.loc[top_protocols]

# Plotting the distribution
plt.figure(figsize=(15, 10))
sns.heatmap(protocol_attack_distribution_top, annot=True, fmt="d", cmap='viridis', cbar_kws={"shrink": .5})
plt.title('Distribution of Attack Types Across Top Protocols')
plt.xlabel('Attack Type')
plt.ylabel('Protocol Type')
plt.show()


In [ ]:
# Count the frequency of each attack type
attack_counts = df['label'].value_counts()
print(attack_counts)


In [ ]:
df.columns.tolist()

In [ ]:
# Analysis 1: Traffic Pattern and Behavior Analysis
plt.figure(figsize=(10, 6))
sns.histplot(df['flow_duration'], bins=50, kde=False)
plt.title('Flow Duration Distribution')
plt.xlabel('Flow Duration')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Analysis 2: Network Activity Overview
plt.figure(figsize=(10, 6))
sns.countplot(x='Number', data=df)
plt.title('Network Activity Overview')
plt.xlabel('Number of Events')
plt.ylabel('Count')
plt.show()

In [ ]:
# Analysis 3: Relationship and Dependency Analysis
plt.figure(figsize=(10, 6))
sns.scatterplot(x='flow_duration', y='Rate', hue='label', data=df, palette='viridis')
plt.title('Flow Duration vs Rate')
plt.xlabel('Flow Duration')
plt.ylabel('Rate')
plt.show()

In [ ]:
# Analysis 4: Network Traffic Significance
plt.figure(figsize=(10, 6))
sns.barplot(x='label', y='Weight', data=df)
plt.title('Network Traffic Significance')
plt.xlabel('Traffic Type')
plt.ylabel('Weight')
plt.show()

In [ ]:
# Analysis 5: Network Variability and Anomalies
plt.figure(figsize=(10, 6))
sns.boxplot(x='label', y='Variance', data=df)
plt.title('Network Variability and Anomalies')
plt.xlabel('Traffic Type')
plt.ylabel('Variance')
plt.show()

In [ ]:

# Analysis 6: Event Magnitude
plt.figure(figsize=(10, 6))
sns.barplot(x='label', y='Magnitue', data=df)
plt.title('Event Magnitude')
plt.xlabel('Traffic Type')
plt.ylabel('Magnitude')
plt.show()

In [ ]:
# Analysis 8: Protocol Usage Analysis
protocol_counts = df['Protocol Type'].value_counts()
plt.figure(figsize=(10, 6))
protocol_counts.plot(kind='bar')
plt.title('Protocol Usage Distribution')
plt.xlabel('Protocol Type')
plt.ylabel('Count')
plt.show()

In [ ]:
# Analysis 9: Flag Analysis
flags = ['fin_flag_number', 'syn_flag_number', 'rst_flag_number']
df_flags = df[flags].sum()
plt.figure(figsize=(10, 6))
df_flags.plot(kind='bar')
plt.title('Flag Analysis')
plt.xlabel('Flag Type')
plt.ylabel('Count')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.histplot(df['Tot size'], bins=50, kde=False)
plt.title('Packet Size Distribution')
plt.xlabel('Total Size')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Additional Analysis: IAT Distribution
plt.figure(figsize=(10, 6))
sns.histplot(df['IAT'], bins=50, kde=False)
plt.title('Inter-Arrival Time Distribution')
plt.xlabel('IAT')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Frequency of each attack type
# attack_counts = df['label'].value_counts()

# plt.figure(figsize=(12, 8))
# attack_counts.plot(kind='bar')
# plt.title('Frequency of Attack Types')
# plt.xlabel('Attack Type')
# plt.ylabel('Frequency')
# plt.show()
plt.figure(figsize=(14, 8))
attack_counts = df['label'].value_counts()
attack_counts.plot(kind='barh')  # Horizontal bar chart for readability
plt.title('Frequency of Attack Types')
plt.xlabel('Frequency')
plt.ylabel('Attack Type')
plt.tight_layout()  # Adjust layout to fit labels
plt.show()

In [ ]:
# Detecting patterns and anomalies in traffic flow and duration
# plt.figure(figsize=(12, 8))
# sns.boxplot(data=df, x='label', y='flow_duration')
# plt.title('Flow Duration by Traffic Type')
# plt.xlabel('Traffic Type')
# plt.ylabel('Flow Duration')
# plt.show()

plt.figure(figsize=(14, 8))
sns.boxplot(data=df, x='label', y='flow_duration')
plt.title('Flow Duration by Traffic Type')
plt.xlabel('Traffic Type')
plt.ylabel('Flow Duration (Log Scale)')
plt.yscale('log')  # Log scale to handle wide range of values
plt.xticks(rotation=45, ha='right')  # Rotate x labels for readability
plt.tight_layout()
plt.show()

In [ ]:
# Detecting anomalies in protocol usage
# plt.figure(figsize=(12, 8))
# sns.countplot(data=df, x='Protocol Type', hue='label')
# plt.title('Protocol Usage by Traffic Type')
# plt.xlabel('Protocol Type')
# plt.ylabel('Count')
# plt.legend(title='Traffic Type')
# plt.show()
#plt.figure(figsize=(14, 8))
# protocol_usage = df.groupby('label')['Protocol Type'].value_counts().unstack().fillna(0)
# protocol_usage.plot(kind='bar', stacked=True)  # Stacked bar chart for protocol usage
# plt.title('Protocol Usage by Traffic Type')
# plt.xlabel('Traffic Type')
# plt.ylabel('Count')
# plt.xticks(rotation=45, ha='right')
# plt.legend(title='Protocol Type', bbox_to_anchor=(1.05, 1), loc='upper left')  # Move legend out of the plot
# plt.tight_layout()
# plt.show()
# plt.figure(figsize=(14, 10))  # Increased figure size
# protocol_counts = df['Protocol Type'].value_counts().nlargest(10)  # Only take the top 10 protocols
# protocol_counts.plot(kind='bar')
# plt.title('Top 10 Protocol Usage by Traffic Type')
# plt.xlabel('Protocol Type')
# plt.ylabel('Count')
# plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better fit
# plt.tight_layout()  # Adjust layout
# plt.show()
# Sum the protocol columns to get the total count for each protocol
protocols = ['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']
protocol_counts = df[protocols].sum().sort_values(ascending=False)

# Now let's plot the protocol usage
plt.figure(figsize=(14, 8))
protocol_counts.plot(kind='bar')
plt.title('Protocol Usage Counts')
plt.xlabel('Protocol')
plt.ylabel('Count')
plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better fit
plt.tight_layout()  # Adjust layout
plt.show()

In [ ]:
# Analyzing flag anomalies that may indicate network vulnerabilities
# flags = ['fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number', 'ack_flag_number']
# df_flags = df[flags].sum()

# plt.figure(figsize=(12, 8))
# df_flags.plot(kind='bar')
# plt.title('Sum of Flags as Potential Vulnerability Indicators')
# plt.xlabel('Flag Type')
# plt.ylabel('Sum')
# plt.show()
plt.figure(figsize=(14, 8))
flags = ['fin_flag_number', 'syn_flag_number', 'rst_flag_number', 'psh_flag_number', 'ack_flag_number']
df_flags = df[flags].sum().sort_values(ascending=False)
df_flags.plot(kind='bar')
plt.title('Sum of Flags as Potential Vulnerability Indicators')
plt.xlabel('Flag Type')
plt.ylabel('Sum')
for index, value in enumerate(df_flags):
    plt.text(index, value, f'{value:.0f}', ha='center', va='bottom')  # Add text labels on bars
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Calculate some key metrics
average_flow_duration = df['flow_duration'].mean()
max_protocol_usage = df[protocols].sum().max()
top_attack_type = df['label'].value_counts().idxmax()

# Create a figure for the scorecard
fig, ax = plt.subplots(figsize=(10, 3))

# Hide the axes
ax.axis('off')

# Set background color
fig.patch.set_facecolor('lightgray')
ax.set_facecolor('lightgray')

# Draw a rectangle as the scorecard background
rect = plt.Rectangle((0.1, 0.1), width=0.8, height=0.8, transform=fig.transFigure, 
                     facecolor='white', zorder=0)
ax.add_patch(rect)

# Display key metrics
bbox_props = dict(boxstyle="round,pad=0.3", ec="black", lw=2, facecolor="white")
ax.text(0.2, 0.6, f'Average Flow Duration:\n{average_flow_duration:.2f} seconds', 
        fontsize=12, bbox=bbox_props)
ax.text(0.2, 0.4, f'Max Protocol Usage:\n{max_protocol_usage}', 
        fontsize=12, bbox=bbox_props)
ax.text(0.2, 0.2, f'Top Attack Type:\n{top_attack_type}', 
        fontsize=12, bbox=bbox_props)

# Show the scorecard
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Example metrics for the scorecard
average_flow_duration = df['flow_duration'].mean()
max_protocol_usage = df[protocols].sum().max()
top_attack_type = df['label'].value_counts().idxmax()

# Create the figure
plt.figure(figsize=(10, 5))
plt.suptitle('Network Traffic Scorecard', fontsize=20)

# Scorecard for Average Flow Duration
plt.subplot(1, 3, 1)
plt.text(0.5, 0.5, f'Avg Flow Duration\n{average_flow_duration:.2f} sec', fontsize=15, 
         ha='center', va='center')
plt.gca().set_axis_off()

# Scorecard for Max Protocol Usage
plt.subplot(1, 3, 2)
plt.text(0.5, 0.5, f'Max Protocol Usage\n{max_protocol_usage}', fontsize=15, 
         ha='center', va='center')
plt.gca().set_axis_off()

# Scorecard for Top Attack Type
plt.subplot(1, 3, 3)
plt.text(0.5, 0.5, f'Top Attack Type\n{top_attack_type}', fontsize=15, 
         ha='center', va='center')
plt.gca().set_axis_off()

# Adjust the layout and display
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()


In [ ]:
# Print a sample of values from the 'Protocol Type' column
print(df['Protocol Type'].sample(10))


In [ ]:
protocol_http_crosstab = pd.crosstab(df['Protocol Type'], df['HTTP'])

print(protocol_http_crosstab)

In [ ]:
# 1.1 Data Label Analysis
attack_counts = df['label'].value_counts()
print(attack_counts)

# 1.2 Attack Characterization
attack_characteristics = df.groupby('label').mean()
print(attack_characteristics)

# 1.3 Impact Assessment (example using flow duration to quantify impact)
attack_impact = df.groupby('label')['flow_duration'].mean().sort_values(ascending=False)
print(attack_impact)

# Visualizing attack frequencies
plt.figure(figsize=(10, 8))
sns.barplot(y=attack_counts.index, x=attack_counts.values)
plt.title('Frequency of Attack Types')
plt.xlabel('Counts')
plt.ylabel('Attack Type')
plt.show()

In [ ]:
from sklearn.ensemble import IsolationForest

# 2.1 Traffic Flow Analysis
# Selecting relevant features for traffic flow analysis
traffic_features = df[['flow_duration', 'Rate', 'Srate', 'Drate']]

# Plotting distributions for traffic features
for feature in traffic_features:
    plt.figure(figsize=(6, 4))
    sns.histplot(df[feature], bins=50, kde=True)
    plt.title(f'Distribution of {feature}')
    plt.show()

# 2.2 Anomaly Detection
# Fitting the Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
anomalies = iso_forest.fit_predict(traffic_features)
df['anomaly'] = anomalies

# Counting anomalies
anomaly_counts = pd.value_counts(df['anomaly'])
print(anomaly_counts)

# Visualizing the anomalies
sns.scatterplot(data=df, x='flow_duration', y='Rate', hue='anomaly')
plt.title('Anomalies in IoT Traffic')
plt.show()

# 2.3 Protocol Anomalies
protocol_usage = df['Protocol Type'].value_counts()
print(protocol_usage)


In [ ]:
# 3.1 Network Architecture Analysis
# Analyzing different protocols for vulnerabilities
protocol_analysis = df.groupby('Protocol Type').agg({'flow_duration': 'mean', 'Header_Length': 'mean'})
print(protocol_analysis)

# 3.2 Communication Protocol Analysis
# Ensure that 'Protocol Type' is treated as a categorical variable
df['Protocol Type'] = df['Protocol Type'].astype('category')

# Distribution of header lengths across different protocols
plt.figure(figsize=(10, 8))
sns.boxplot(x='Protocol Type', y='Header_Length', data=df)
plt.title('Header Length Distribution by Protocol Type')
plt.xticks(rotation=90)  # Rotate x labels for better readability if necessary
plt.show()


# 3.3 Vulnerability Identification
# Combining attack patterns with protocol usage to infer vulnerabilities
vulnerabilities = df[df['anomaly'] == -1]['Protocol Type'].value_counts()
print(vulnerabilities)


In [ ]:
from sklearn.ensemble import IsolationForest
# Selecting relevant features for traffic flow analysis
traffic_features = df[['flow_duration', 'Rate', 'Srate', 'Drate']]

# Plotting distributions for traffic features
for feature in traffic_features.columns:
    sns.histplot(df[feature], bins=50, kde=True)
    plt.title(f'Distribution of {feature}')
    plt.show()

# Fitting the Isolation Forest model
iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
anomalies = iso_forest.fit_predict(traffic_features)
df['anomaly'] = anomalies

# Counting anomalies
anomaly_counts = pd.value_counts(df['anomaly'])
print(anomaly_counts)

# Visualizing the anomalies
# Ensure that 'df' is replaced with the actual DataFrame variable name, which is 'data' in this context
sns.scatterplot(data=df, x='flow_duration', y='Rate', hue='anomaly')
plt.title('Anomalies in IoT Traffic')
plt.show()

In [ ]:
# Analyzing TCP flags by protocol type
tcp_flags = ['syn_flag_number', 'ack_flag_number', 'fin_flag_number']
protocol_flag_counts = df.groupby('Protocol Type')[tcp_flags].sum()

# Plotting the results
protocol_flag_counts.plot(kind='bar', figsize=(12, 6))
plt.title('TCP Flag Counts by Protocol Type')
plt.xlabel('Protocol Type')
plt.ylabel('Flag Count')
plt.show()


In [ ]:
# Analyzing flow duration by protocol type
avg_flow_duration_by_protocol = df.groupby('Protocol Type')['flow_duration'].mean()

# Plotting the results
plt.figure(figsize=(10, 6))
avg_flow_duration_by_protocol.plot(kind='bar')
plt.title('Average Flow Duration by Protocol Type')
plt.xlabel('Protocol Type')
plt.ylabel('Average Flow Duration')
plt.show()


In [ ]:
# Analyzing network traffic types by protocol type
traffic_types = ['HTTP', 'HTTPS', 'DNS', 'SSH']
protocol_traffic_counts = df.groupby('Protocol Type')[traffic_types].sum()

# Plotting the results
protocol_traffic_counts.plot(kind='bar', figsize=(12, 6))
plt.title('Network Traffic Type Counts by Protocol Type')
plt.xlabel('Protocol Type')
plt.ylabel('Traffic Type Count')
plt.show()


In [ ]:

protocol_columns = ['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 
                    'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']

# Calculating the sum (or count) for each protocol
protocol_counts = df[protocol_columns].sum()

# Plotting the distribution of different protocols
plt.figure(figsize=(14, 8))
protocol_counts.plot(kind='bar')
plt.title('Distribution of Different Protocols in Network Traffic')
plt.xlabel('Protocol Type')
plt.ylabel('Count')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.show()


In [ ]:
protocol_flags = ['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']

# Calculate the sum for each protocol flag to see their distribution
protocol_sum = df[protocol_flags].sum().sort_values(ascending=False)

# Plot the protocol distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=protocol_sum.index, y=protocol_sum.values)
plt.title('Distribution of Protocol Flags in Network Traffic')
plt.xlabel('Protocol Flags')
plt.ylabel('Sum')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.show()


In [ ]:
# Assuming HTTP traffic is indicated by a binary flag where 1 means HTTP traffic is present
http_flows = df[df['HTTP'] == 1]  # Filter flows where HTTP traffic is present

# Calculate the average flow duration for HTTP traffic
avg_http_flow_duration = http_flows['flow_duration'].mean()

print(f'Average Flow Duration for HTTP Traffic: {avg_http_flow_duration}')

# You can repeat this process for other protocols as well


In [ ]:
# Melting the DataFrame to have protocol flags as a single categorical variable
melted_df = df.melt(id_vars=['flow_duration'], value_vars=protocol_flags, var_name='Protocol', value_name='Flag')

# Filter out only the rows where the protocol flag is present (i.e., Flag == 1)
melted_df = melted_df[melted_df['Flag'] == 1]

# Plotting the flow duration distribution for different protocols
plt.figure(figsize=(14, 8))
sns.boxplot(x='Protocol', y='flow_duration', data=melted_df)
plt.title('Flow Duration Distribution by Protocol')
plt.xlabel('Protocol')
plt.ylabel('Flow Duration')
plt.xticks(rotation=90)  # Rotate x-axis labels for better readability
plt.yscale('log')  # Using a log scale due to wide range of durations
plt.show()


In [ ]:
print(df.columns)

In [ ]:
# List of corrected TCP flag columns based on DataFrame
flag_columns = [
    'fin_flag_number', 'syn_flag_number', 'rst_flag_number',
    'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 'cwr_flag_number'
]

# Calculate the sum for each flag column to see their distribution
flag_sum = df[flag_columns].sum()

# Plot the flag distribution
plt.figure(figsize=(12, 6))
sns.barplot(x=flag_sum.index, y=flag_sum.values)
plt.title('Distribution of TCP Flag Counts in Network Traffic')
plt.xlabel('TCP Flags')
plt.ylabel('Sum')
plt.show()


From the "Distribution of TCP Flag Counts in Network Traffic" plot, it looks like the ack_flag_number is the most prevalent, followed by syn_flag_number and fin_flag_number. This distribution is typical as ACK and SYN flags are commonly used in the establishment and acknowledgment of TCP connections.


The high count of certain flags might warrant a closer look to see if they correlate with specific types of attacks or benign traffic patterns.

In [ ]:
# For the flow duration analysis, we'll use the binary protocol columns
protocol_flags = ['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']

# Calculate the average flow duration for when each protocol is present
avg_flow_duration_by_protocol = {protocol: df[df[protocol] == 1]['flow_duration'].mean() for protocol in protocol_flags}

# Convert the dictionary to a DataFrame for easy plotting
avg_flow_duration_df = pd.DataFrame.from_dict(avg_flow_duration_by_protocol, orient='index', columns=['Average Flow Duration'])

# Plot the average flow duration for each protocol
avg_flow_duration_df.sort_values('Average Flow Duration', ascending=False).plot(kind='bar', figsize=(12, 6))
plt.title('Average Flow Duration by Protocol')
plt.xlabel('Protocol')
plt.ylabel('Average Flow Duration')
plt.show()


The "Average Flow Duration by Protocol" plot indicates ARP traffic has the highest average flow duration, followed by DNS traffic. This could suggest that ARP and DNS flows tend to have longer durations within the dataset. The longer flow durations for ARP might be due to ARP spoofing or poisoning attacks, which are common in IoT environments, while DNS could be related to DNS tunneling or reconnaissance activities.

Longer flow durations for protocols like ARP and DNS might suggest unusual network behavior, potentially indicative of malicious activity.

Network security professionals can prioritize which protocols to scrutinize for vulnerabilities or anomalous behavior. If ARP and DNS are showing unusual patterns, they might need more robust security measures or monitoring strategies.

In [ ]:
attack_counts = df['label'].value_counts()


In [ ]:
# Count the frequency of each attack type
attack_type_counts = df['label'].value_counts().reset_index()
attack_type_counts.columns = ['Attack Type', 'Frequency']

# Export this data to a CSV file for Tableau
attack_type_counts.to_csv('attack_type_counts.csv', index=False)

# Additionally, you can calculate the average impact of each attack type
# For example, using flow_duration as a proxy for impact
attack_impact = df.groupby('label')['flow_duration'].mean().reset_index()
attack_impact.columns = ['Attack Type', 'Average Impact']

# Export impact data to CSV
attack_impact.to_csv('attack_impact.csv', index=False)

In [ ]:
features = df[['flow_duration', 'Rate', 'Srate', 'Drate']]
from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(n_estimators=100, contamination='auto', random_state=42)
df['anomaly'] = iso_forest.fit_predict(features)

plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='flow_duration', y='Rate', hue='anomaly', palette='Set1')
plt.title('Anomaly Detection in IoT Traffic')
plt.xlabel('Flow Duration')
plt.ylabel('Rate')
plt.legend(title='Anomaly')
plt.show()

In [ ]:
#Correlation Analysis
correlation_matrix = df[['flow_duration', 'Rate', 'Srate', 'Drate']].corr()
sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Matrix of IoT Traffic Features')
plt.show()


In [ ]:
#Clustering
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=5, random_state=42)
df['cluster'] = kmeans.fit_predict(features)
sns.scatterplot(data=df, x='flow_duration', y='Rate', hue='cluster', palette='Set1')
plt.legend(title='Cluster')
plt.show()


In [ ]:
#Feature Importance
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
rf.fit(features, df['anomaly'])
importances = rf.feature_importances_
plt.barh(features.columns, importances)
plt.title('Feature Importances for Anomaly Detection')
plt.show()


In [ ]:
#Multivariate Analysis
pairplot = sns.pairplot(df[['flow_duration', 'Rate', 'Srate', 'Drate', 'anomaly']], hue='anomaly')
pairplot.fig.suptitle('Pairplot of Features with Anomaly Highlighted', y=1.02)
plt.show()


In [ ]:
#Protocol Behavior Analysis

protocols = ['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']

protocol_anomaly_rates = {}

for protocol in protocols:

    anomaly_column = protocol + '_anomaly'
    df[anomaly_column] = df.apply(lambda x: 1 if x[protocol] > 0 and x['anomaly'] == -1 else 0, axis=1)
    

    protocol_anomaly_rates[protocol] = df[anomaly_column].mean()


protocol_anomaly_rates_df = pd.DataFrame(list(protocol_anomaly_rates.items()), columns=['Protocol', 'Anomaly Rate'])


protocol_anomaly_rates_df.sort_values('Anomaly Rate', ascending=False, inplace=True)


protocol_anomaly_rates_df


In [ ]:
# Get a quick overview of the dataset
print(df.info())
print(df.head())
print(df.describe())

# If 'Protocol Type' contains unclear integer values, explore its unique values
print(df['Protocol Type'].unique())

In [ ]:
protocol_types = df['Protocol Type'].unique()

# Checking the 'label' column for understanding the classification of data
labels = df['label'].unique()

(protocol_types, labels)

In [ ]:
# Sum of each flag type
flags_sum = df[['fin_flag_number', 'syn_flag_number','rst_flag_number', 'psh_flag_number', 'ack_flag_number','ece_flag_number', 'cwr_flag_number']].sum()

# Average flow duration for each type of attack
avg_flow_duration = df.groupby('label')['flow_duration'].mean()

# Rate and count of different protocols
protocol_rates = df[['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']].mean()
protocol_counts = df[['HTTP', 'HTTPS', 'DNS', 'Telnet', 'SMTP', 'SSH', 'IRC', 'TCP', 'UDP', 'DHCP', 'ARP', 'ICMP', 'IPv', 'LLC']].sum()


flags_sum, avg_flow_duration, protocol_rates, protocol_counts

In [ ]:
# Calculate the sum of each TCP flag type
flags_sum = df[
    ['fin_flag_number', 'syn_flag_number', 'rst_flag_number',
     'psh_flag_number', 'ack_flag_number', 'ece_flag_number', 
     'cwr_flag_number']
].sum()

# Sort the flag sums for better visualization
flags_sum_sorted = flags_sum.sort_values(ascending=False)

print(flags_sum_sorted)


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the aesthetic style of the plots
sns.set_style("whitegrid")

# Plot for TCP Flags Analysis
plt.figure(figsize=(14, 7))
flags_sum.plot(kind='bar', color='skyblue')
plt.title('Total of TCP Flag Counts')
plt.ylabel('Total Flags')
plt.xlabel('Flag Type')
plt.xticks(rotation=45)
plt.tight_layout()  # Adjusts plot to ensure everything fits without overlapping
plt.show()

# Plot for Average Flow Duration by Attack Type
# We sort the values for better visualization and take the top 20 attack types for a cleaner plot
top_attack_types = df.groupby('label')['flow_duration'].mean().sort_values(ascending=False)[:20]
plt.figure(figsize=(14, 7))
top_attack_types.plot(kind='bar', color='salmon')
plt.title('Average Flow Duration by Attack Type')
plt.ylabel('Average Flow Duration')
plt.xlabel('Attack Type')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Plot for Protocol Rates
plt.figure(figsize=(14, 7))
protocol_rates.plot(kind='bar', color='lightgreen')
plt.title('Average Rate by Protocol')
plt.ylabel('Average Rate')
plt.xlabel('Protocol')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

# Plot for Protocol Counts
plt.figure(figsize=(14, 7))
protocol_counts.plot(kind='bar', color='orchid')
plt.title('Total Counts by Protocol')
plt.ylabel('Total Counts')
plt.xlabel('Protocol')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()
